# 傾向スコアマッチング

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from psmpy import PsmPy
from psmpy.functions import cohenD

import gpboost as gpb
# https://gpboost.readthedocs.io/en/latest/pythonapi/gpboost.GPModel.html

from patsy import dmatrix

In [5]:
filepath = '../../../data/processed/df_filtered_5years.xlsx'
df = pd.read_excel(filepath)
df['log_income'] = np.log(df['income'])
df = df.dropna()
df.head()

,island,year,island_id,region_code,region_name,prefecture_code,prefecture_code.1,population,dummy_island_has_bridge,dummy_island_is_connected_mainland,year_bridge_opened,dummy_after_bridge_opened,year_connect_mainland,dummy_after_connect_mainland,income,JPNCPIALLAINMEI,nominal_income,log_income
2,島後,2005,0,32528,隠岐の島町,32,島根県,17259.0,0,0,0,0,0,0,196868.811077,96.93730,19083931.0,12.190293
3,島後,2010,0,32528,隠岐の島町,32,島根県,15930.0,0,0,0,0,0,0,166610.045283,96.53008,16082881.0,12.023411
4,島後,2015,0,32528,隠岐の島町,32,島根県,14901.0,0,0,0,0,0,0,161353.170000,100.00000,16135317.0,11.991351
5,島後,2020,0,32528,隠岐の島町,32,島根県,13882.0,0,0,0,0,0,0,171650.091455,101.79860,17473739.0,12.053213
6,中ノ島,1985,1,32525,海士町,32,島根県,3339.0,0,0,0,0,0,0,30999.550430,85.34827,2645758.0,10.341728


## 処置変数と共変量の指定

## ロジスティック回帰モデルの適用

In [15]:
group = df[['island_id', 'year']]
treatment = df['dummy_after_bridge_opened']
X = dmatrix('population + log_income', data=df, return_type='dataframe')

model = gpb.GPModel(group_data=group, likelihood='bernoulli_logit') # ベルヌーイ分布のロジットリンク関数
model.fit(y=treatment, X=X, params={'std_dev': True})

pred = model.predict(X_pred=X, group_data_pred=group)['mu']
residuals = treatment - pred

print(model.summary())

Model summary:
 Log-lik    AIC    BIC
 -102.12 214.24 236.31
Nb. observations: 611
Nb. groups: 154 (island_id), 8 (year)
-----------------------------------------------------
Covariance parameters (random effects):
               Param.
island_id  22470.7943
year        5509.3007
-----------------------------------------------------
Linear regression coefficients (fixed effects):
              Param.  Std. dev.  z value  P(>|z|)
Intercept  -151.0083    17.3563  -8.7005   0.0000
population    0.0074     0.0011   6.8489   0.0000
log_income    3.7812     1.0157   3.7228   0.0002


## 傾向スコアの予測

In [19]:
propensity_scores = model.predict(X_pred=X, group_data_pred=group)['mu']

In [30]:
# 逆確率重み付け（IPW）の計算
weights = treatment / propensity_scores + (1 - treatment) / (1 - propensity_scores)

# 重み付けを用いた平均処置効果（ATE）
outcome = df['population']
weighted_outcome_treatment = np.sum(weights * treatment * outcome) / np.sum(weights * treatment)
weighted_outcome_control = np.sum(weights * (1 - treatment) * outcome) / np.sum(weights * (1 - treatment))

ATE = weighted_outcome_treatment - weighted_outcome_control

print(f'ATE: {ATE:.2f}')

ATE: 4837.24


In [36]:
import numpy as np

# ブートストラップサンプルの数
n_bootstraps = 1000
bootstrapped_ates = []

# ブートストラップサンプルを生成してATEを計算
for _ in range(n_bootstraps):
    # ブートストラップサンプルを生成
    bootstrap_sample = df.sample(n=len(df), replace=True)
    
    # 傾向スコアの再計算
    group = bootstrap_sample[['island_id', 'year']]
    treatment = bootstrap_sample['dummy_after_bridge_opened']
    X = dmatrix('population + log_income', data=bootstrap_sample, return_type='dataframe')
    
    model = gpb.GPModel(group_data=group, likelihood='bernoulli_logit')
    model.fit(y=treatment, X=X, params={'std_dev': True})
    propensity_scores = model.predict(X_pred=X, group_data_pred=group)['mu']
    
    # 逆確率重み付け（IPW）の計算
    weights = treatment / propensity_scores + (1 - treatment) / (1 - propensity_scores)
    
    # 重み付けを用いた平均処置効果（ATE）の計算
    outcome = bootstrap_sample['population']
    weighted_outcome_treatment = np.sum(weights * treatment * outcome) / np.sum(weights * treatment)
    weighted_outcome_control = np.sum(weights * (1 - treatment) * outcome) / np.sum(weights * (1 - treatment))
    
    bootstrap_ate = weighted_outcome_treatment - weighted_outcome_control
    bootstrapped_ates.append(bootstrap_ate)

# ATEの標準誤差を計算
standard_error = np.std(bootstrapped_ates)

# ATEの信頼区間を計算
lower_bound = np.percentile(bootstrapped_ates, 2.5)
upper_bound = np.percentile(bootstrapped_ates, 97.5)

print(f'ATE: {ATE:.2f}')
print(f'Standard Error: {standard_error:.2f}')
print(f'95% Confidence Interval: [{lower_bound:.2f}, {upper_bound:.2f}]')

ATE: 4837.24
Standard Error: 755.87
95% Confidence Interval: [3737.45, 6722.59]


## 結果

ATE: 4837.24

Standard Error: 755.87

95% Confidence Interval: [3737.45, 6722.59]